# CS 5683: Project 2
Name: Pankajdeer Bikumalla <br>

CWID: A20220181

# Project-2: Locality Sensitive Hashing

In [ ]:
# importing required libraries
import random
import os
import numpy as np
import re

from random import randrange
from shutil import copyfile

## Execute the follwing two cells to generate your data sample

In [ ]:
# MY_ID should be string
MY_ID = '181'
# Example MY_ID='819'

n_1 = 0
n_2 = 0
n_3 = 0
task_dict = {0:'taska',1:'taskb',2:'taskc',3:'taskd',4:'taske'}
# id_dict = {0:n_1,1:n_2,2:n_3}

try:
    n_1 = int(MY_ID[-1]) % 5
    n_2 = int(MY_ID[-2]) % 5
    n_3 = int(MY_ID[-3]) % 5
    
    while n_1 == n_2 or n_1 == n_3:
        n_1 = (n_1 + 1) % 5
    
    while n_1 == n_2 or n_2 == n_3:
        n_2 = (n_2 + 1) % 5

except Exception as e:
    print('Please enter a valid ID...')
id_dict = {0:n_1,1:n_2,2:n_3}        

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#professor give ($)
data_path = '/content/drive/My Drive/3Sem/CS5683/Projects/Project 2/corpus-20090418' 

# Edit this path if the data directory is not in the current directory
try:
    os.makedirs('Data_Sample')
    os.makedirs('Original_Sample')
except Exception as e:
    pass

for i in range(3):
    task = task_dict[id_dict[i]]
    
    for file in os.listdir(data_path):
        if task in file:
            if 'orig' in file:
                copyfile(data_path + '/' + file, 'Original_Sample/' + file)
            else:
                copyfile(data_path + '/' + file, 'Data_Sample/' + file)
                

Your dataset for this project will be in <b>Data_Sample</b>


Your query choices will be in <b>Original_Sample</b> directory


You have to use any one original Wikipedia article from <b>Original_Sample</b> for <b>Fact Check</b> steps

### STEP - 1: Shingling (20 points)

In [ ]:
# converted upper case charaters to lower case
prefix = '/content/Data_Sample/'
files_list=os.listdir(prefix)
# Dict stores, [Keys: document],[values: words in each documents]
Dict = {}
combined = []
for a in files_list:
  data = open((prefix+a),'r',encoding ='utf8',errors='ignore').read()
  data_words= data.lower().split()
  Dict[a]=data_words


In [ ]:
# Function to get count of unique shingles (based on k values)
# this function returns both count of unique shingles and set(unique shingles) for given values of k

def count_shingles(Dict,k,files):
  count_shingles = 0
  unique_shingles=set()  
  for key in files:
    shingles = []
    for i in range(len(Dict[key])-k+1):
      shingles=Dict[key][i:i+k]
      shingles = ' '.join(shingles)
      if shingles in unique_shingles:
        continue
      else:
        unique_shingles.add(shingles)
        count_shingles=count_shingles+1
  return count_shingles, unique_shingles

In [ ]:
#count of 3 shingles
count_3shingles,unique_3shingles = count_shingles(Dict,3,os.listdir(prefix))
print("number of unique 3-shingles count is ", count_3shingles)

#count of 4 shingles
count_4shingles, unique_4shingles = count_shingles(Dict,4,os.listdir(prefix))
print("number of unique 4-shingles count is ", count_4shingles)

#count of 4 shingles
count_5shingles, unique_5shingles = count_shingles(Dict,5,os.listdir(prefix))
print("number of unique 5-shingles count is ", count_5shingles)

number of unique 3-shingles count is  7929
number of unique 4-shingles count is  8721
number of unique 5-shingles count is  9120


Report results (number of unique k-shingles) for k={3,4,5} below:
1. k=3: 7929
2. k=4: 8721
3. k=5: 9120

In [ ]:
# Type your code to get the 5-shingle index here

count_5shingles, unique_5shingles = count_shingles(Dict,5,os.listdir(prefix))
#shingles are indexed here from {0 to count_5_uniqueshingles}
unique_5shingles_list = list(unique_5shingles)

# Storing the unique shingles by document wise
dict_5shingles = {}
k = 5
for file in Dict:
  file_5shingles=set()
  for i in range(len(Dict[file]) - k + 1): 
    shingles_5 = Dict[file][i:i + k]
    shingles_5 = ' '.join(shingles_5)
    if shingles_5 in file_5shingles:
      continue
    else:
      file_5shingles.add(shingles_5)
  dict_5shingles[file]=file_5shingles

### STEP - 2: Min-Hashing (40 points)

In [ ]:
# ***************************************************NEW***********************************************************
# Generate Hash functions - 
    # We use (ax + b) mod N formula to permute shingle index
    # where a,b are random numbers, N total index size, and x is the index
# We need to do L permutations - In other words we need to have L permutations (lists) of new indexes
# Following function takes total index size N and L as arguments
    # And returns L new lists of size N
    
def get_hash_functions(N,L):
    hash_functions = []
    
    for itr in range(L):
        a=randrange(1,400)
        b=randrange(1,400)
        
        new_hash_function = []
        for i in range(N):
            new_hash_function.append((a * i + b) % N)
        
        hash_functions.append(new_hash_function)
    return hash_functions


### 2a: Generating Hash Functions

In [ ]:
# Type your code here to generate all L hash functions
# Generate hash functions only for shingle index created for k=5
N =count_5shingles

L50 = 50
H50 = get_hash_functions(N,L50)

L100 = 100
H100 = get_hash_functions(N,L100)

L200 = 200
H200 = get_hash_functions(N,L200)

L500 = 500
H500 = get_hash_functions(N,L500)

L1000 = 1000
H1000 = get_hash_functions(N,L1000)


In [ ]:
#for initializing values to infinity
import math

### 2c. Generating Signature Matrix

In [ ]:
# Function to generate: signature matrix
#uni_shingles: unique 5 singles
#dict_doc_shingles: [keys: file names],[values:shingles in respective files]

def sigMatrix(H,uni_shingles,dict_doc_shingles):
  list_values = []
  for f in dict_doc_shingles:
    values_mat = [math.inf for i in range(len(H))] 
    col = 0
    for s in uni_shingles:
      if s in list(dict_doc_shingles[f]):
        for row in range(len(H)):
          if H[row][col] < values_mat[row]:
            values_mat[row] = H[row][col]
      # dict_columns[f]=col
      col=col+1
    list_values.append(values_mat)
  sig_mat = np.mat(np.array(list_values).T)
  return sig_mat

In [ ]:
# Signature matrices
# Signature matrix: H50
sig_mat_50 = sigMatrix(H50, list(unique_5shingles),dict_5shingles)

# Signature matrix: H100
sig_mat_100 = sigMatrix(H100, list(unique_5shingles),dict_5shingles)

# Signature matrix: H200
sig_mat_200 = sigMatrix(H200, list(unique_5shingles),dict_5shingles)

# Signature matrix: H500
sig_mat_500 = sigMatrix(H500, list(unique_5shingles),dict_5shingles)

# Signature matrix: H1000
sig_mat_1000 = sigMatrix(H1000, list(unique_5shingles),dict_5shingles)

### 2d: Fact check 
Jaccard similarity score greater than t (t=0.85) <br>
<br>
I have conducted analysis by considering the first file in the "Original_sample" folder, for computing jaccard similarity

In [ ]:
# Type your code here to do the fact check 
#      with any one query document in the 'Original_Sample' directory
prefix_orig = '/content/Original_Sample/'
orig_files=os.listdir(prefix_orig)
Dict_orig = {}

#picking first file in original data folder for fat check
data1 = open((prefix_orig+orig_files[0]),'r',encoding ='utf8',errors='ignore').read()
orig_words= data1.lower().split()
Dict_orig[orig_files[0]]=orig_words

# STEP-1: Generate 5-shingles 
    # (if any shingles are not present in your shingle index, simply ignore them)
first_file =[]
first_file.append(os.listdir(prefix_orig)[0])

count_5orig_shingles,unique_5orig_shingles= count_shingles(Dict_orig,5,first_file)

#ignoring shingles which are not in shingle index
lis_orig_5shignles=[]
for shingle in list(unique_5orig_shingles):
  if shingle in list(unique_5shingles):
    lis_orig_5shignles.append(shingle)
  else:
    continue

dict_5orig_shingles = {}
dict_5orig_shingles[orig_files[0]]=set(lis_orig_5shignles)

# STEP-2: Generate signature vector from L hash functions

#Hash 50
sig_origmat_50=sigMatrix(H50, list(unique_5shingles),dict_5orig_shingles)

#Hash 100
sig_origmat_100=sigMatrix(H100, list(unique_5shingles),dict_5orig_shingles)

#Hash 200
sig_origmat_200=sigMatrix(H200, list(unique_5shingles),dict_5orig_shingles)

#Hash 500
sig_origmat_500=sigMatrix(H500, list(unique_5shingles),dict_5orig_shingles)

#Hash 1000
sig_origmat_1000=sigMatrix(H1000, list(unique_5shingles),dict_5orig_shingles)

In [ ]:
# Function: to Compute jaccard similarity
#Files: list of files to be compared for computing jacc similarity
# matrix: signature matrix
#matrix_orig: signature matrix of document from Origanl_sample folder
def jaccordsim(files,H,matrix,matrix_orig):
  jac_list = []
  for i in range(len(files)):
    match_count = 0
    for j in range(len(H)):
      if matrix_orig[j,0]==matrix[j,i]:
        match_count = match_count+1
    jac_sim = (match_count/len(H))
    jac_list.append(jac_sim)
  return jac_list

In [ ]:
# function: to filter documents with jaccard similarity > 0.2
# jac_list:is obtained from jaccordsim() function
def filter_jaclist(jac_list,files,t):
  jac_final = {}
  for i,doc in zip(jac_list,files):
    if i >=t:
      jac_final[doc] = i
  return jac_final

In [ ]:
# STEP-3: Calculate Jaccard similarity of signature vector of orginal doc.
    # and all other documents     
#initializing the jaccard similarity cutoff value
t = 0.2


# Calculating Jacard Similarity for 50 hash functions
jac_list_50= jaccordsim(files_list,H50,sig_mat_50,sig_origmat_50)
#filetered documents with jcd>0.2
jac_50_final = filter_jaclist(jac_list_50,files_list,t)

# Calculating Jacard Similarity for 100 hash functions
jac_list_100= jaccordsim(files_list,H100,sig_mat_100,sig_origmat_100)
#filetered documents with jcd>0.2
jac_100_final = filter_jaclist(jac_list_100,files_list,t)

# Calculating Jacard Similarity for 200 hash functions
jac_list_200= jaccordsim(files_list,H200,sig_mat_200,sig_origmat_200)
#filetered documents with jcd>0.2
jac_200_final = filter_jaclist(jac_list_200,files_list,t)

# Calculating Jacard Similarity for 500 hash functions
jac_list_500= jaccordsim(files_list,H500,sig_mat_500,sig_origmat_500)
#filetered documents with jcd>0.2
jac_500_final = filter_jaclist(jac_list_500,files_list,t)

# Calculating Jacard Similarity for 1000 hash functions
jac_list_1000= jaccordsim(files_list,H1000,sig_mat_1000,sig_origmat_1000)
#filetered documents with jcd>0.2
jac_1000_final = filter_jaclist(jac_list_1000,files_list,t)

***************************************************NEW*********************************************************** <br>
For each L = {50,100,200,500,1000}, report all documents (file_names) below 
that have Jaccard similarity > t=0.2

Sort the documents in decreasing order of the Jaccard similarity


In [ ]:
print("Jaccard Similarity for 50 Hash functions :", sorted(jac_50_final.items(),key=lambda x: x[1], reverse=True))
print('\n')
print("Jaccard Similarity for 100 Hash functions :", sorted(jac_100_final.items(),key=lambda x: x[1], reverse=True))
print('\n')
print("Jaccard Similarity for 200 Hash functions :", sorted(jac_200_final.items(),key=lambda x: x[1], reverse=True))
print('\n')
print("Jaccard Similarity for 500 Hash functions :", sorted(jac_500_final.items(),key=lambda x: x[1], reverse=True))
print('\n')

print("Jaccard Similarity for 1000 Hash functions :", sorted(jac_1000_final.items(),key=lambda x: x[1], reverse=True))

Jaccard Similarity for 50 Hash functions : [('g3pA_taskd.txt', 0.82), ('g4pC_taskd.txt', 0.7), ('g2pB_taskd.txt', 0.48), ('g0pC_taskd.txt', 0.36), ('g2pA_taskd.txt', 0.34), ('g0pB_taskd.txt', 0.3), ('g1pA_taskd.txt', 0.26), ('g4pB_taskd.txt', 0.26), ('g2pC_taskb.txt', 0.22), ('g1pA_taskb.txt', 0.2)]


Jaccard Similarity for 100 Hash functions : [('g3pA_taskd.txt', 0.89), ('g4pC_taskd.txt', 0.66), ('g2pB_taskd.txt', 0.43), ('g1pA_taskd.txt', 0.32), ('g0pC_taskd.txt', 0.32), ('g2pA_taskd.txt', 0.31), ('g2pC_taskd.txt', 0.27), ('g4pB_taskd.txt', 0.26), ('g0pB_taskd.txt', 0.24)]


Jaccard Similarity for 200 Hash functions : [('g3pA_taskd.txt', 0.825), ('g4pC_taskd.txt', 0.775), ('g2pB_taskd.txt', 0.47), ('g0pC_taskd.txt', 0.405), ('g2pA_taskd.txt', 0.33), ('g4pB_taskd.txt', 0.295), ('g0pB_taskd.txt', 0.25), ('g1pA_taskd.txt', 0.245), ('g2pC_taskd.txt', 0.215)]


Jaccard Similarity for 500 Hash functions : [('g3pA_taskd.txt', 0.806), ('g4pC_taskd.txt', 0.702), ('g2pB_taskd.txt', 0.478), ('g

### STEP - 3: LSH (30 points)

In [ ]:
# Type your code here to hash signature matrix into B buckets
# Use the technique to split the signature matrix into b bands of r rows
# Convert only the signature matrix generated with L=1000

b = 50
r = 20
B = 199

In [ ]:
import collections
from collections import defaultdict

### 3a. Hash Signature Matrix

In [ ]:
# Type your code here to do generate candidate documents
# Follow all steps from STEP - 2 fact check (except the Jaccard similarity part)

# STEP - 1: Split your original document signature vector into b bands of r rows
sig_mat_arr = np.array(sig_mat_1000).reshape(b,r,-1)

# generating a values - based on the condition mentioned
a = random.sample(list(np.arange(start = 1,stop=r+1,step =1)),r)


# STEP - 2: Hash using the same hash functions created for 
    # signature matrix hashing (in the previous cell)
bucket = {}
for i in range(sig_mat_arr.shape[0]):
  Dict_hash=collections.defaultdict(list)
  for k in range(sig_mat_arr.shape[2]):
    h =np.sum(a*(sig_mat_arr[i,:,k])) % B
    Dict_hash[h].append(files_list[k])
  bucket.update(Dict_hash)

sig_origmat_1000_arr =np.array(sig_origmat_1000).reshape(50,20,-1)


### 3b. Fact Check

In [ ]:
# Type your code here to do the fact check
t=0.2
# Getting all the candidate items 
cand_items=set()
for i in range(sig_origmat_1000_arr.shape[0]):
  h =np.sum(a*(sig_origmat_1000_arr[i,:,0])) % B
  for key,value in bucket.items():
    if key ==h:
      for j in value:
        cand_items.add(j)

k = 5
cand_list = list(cand_items)
#storing shingles in cand_list document wise
dict_5shingles_3b = {}
for file in cand_list:
  dict_5shingles_3b[file] = dict_5shingles[file]


# Signature matrix: H1000_3b
sig_mat_1000_3b = sigMatrix(H1000, list(unique_5shingles),dict_5shingles_3b)


# Calculating Jacard Similarity for 1000 hash functions
jac_list_1000_3b= jaccordsim(cand_list,H1000,sig_mat_1000_3b,sig_origmat_1000)
#filetered documents with jcd>0.2
jac_1000_final_3b = filter_jaclist(jac_list_1000_3b,cand_list,t)

In [88]:
# Obtaining False positives and false negatives based on the reference from project-2 document
## False Positives
FP=[]
for file in cand_list:
  if file in list(jac_1000_final_3b.keys()):
    continue
  else: 
    FP.append(file)

## False Negatives
FN=[]
for file in list(jac_1000_final.keys()):
  if file in list(jac_1000_final_3b.keys()):
    continue
  else: 
    FN.append(file)

In [ ]:
# Obtaining False positives and false negatives based on the reference from project-2 document
## False Positives
FP=[]
for file in cand_list:
  if file in list(jac_1000_final_3b.keys()):
    continue
  else: 
    FP.append(file)

## False Negatives
FN=[]
for file in list(jac_1000_final.keys()):
  if file in list(jac_1000_final_3b.keys()):
    continue
  else: 
    FN.append(file)

Report all documents (file_names) below that have Jaccard similarity > t=0.2
Sort the documents in decreasing order of the Jaccard similarity


In [ ]:
print("Jaccard Similarity for 1000 Hash functions with t>=0.2 are : ", "\n")
print( sorted(jac_1000_final_3b.items(),key=lambda x: x[1], reverse=True))

Jaccard Similarity for 1000 Hash functions with t>=0.2 are :  

[('g3pA_taskd.txt', 0.814), ('g4pC_taskd.txt', 0.702), ('g0pC_taskd.txt', 0.369), ('g2pA_taskd.txt', 0.287), ('g1pA_taskd.txt', 0.268), ('g4pB_taskd.txt', 0.245), ('g0pB_taskd.txt', 0.203)]


Report the list of false positives and false negatives below



In [ ]:
print(" Number of Flase positives are :", len(FP))
print(" Documents which are Flase positives are :", FP)
print('\n')

print(" Number of Flase Negatives are :", len(FN))
print(" Documents which are Flase Negatives are :", FN)


 Number of Flase positives are : 28
 Documents which are Flase positives are : ['g0pD_taskd.txt', 'g4pE_taskb.txt', 'g2pB_taskc.txt', 'g0pB_taskb.txt', 'g2pA_taskc.txt', 'g1pB_taskb.txt', 'g3pB_taskc.txt', 'g2pC_taskc.txt', 'g2pA_taskb.txt', 'g4pE_taskd.txt', 'g0pD_taskc.txt', 'g4pD_taskb.txt', 'g4pE_taskc.txt', 'g4pB_taskb.txt', 'g3pA_taskc.txt', 'g0pE_taskc.txt', 'g0pC_taskb.txt', 'g1pD_taskb.txt', 'g4pC_taskb.txt', 'g2pC_taskb.txt', 'g1pB_taskc.txt', 'g2pE_taskc.txt', 'g3pB_taskb.txt', 'g2pE_taskd.txt', 'g0pE_taskb.txt', 'g3pC_taskc.txt', 'g4pC_taskc.txt', 'g0pB_taskc.txt']


 Number of Flase Negatives are : 1
 Documents which are Flase Negatives are : ['g2pB_taskd.txt']


### BONUS : 10 points

To perform the experiment in the bonus task section, I have created a function which takes the signature_matrix computed in the previous steps by inheritence

In [ ]:
# Experiment with different values of b,r,B, and t
    # to reduce the number of false positives and false negatives
    # Report all results in a table in a separate word document


#### For different combinations of b(bands),r(rows),B(bucket):
the following function return the count of false positives and false negatives

In [ ]:
# Function which can give us output of false posities, negatives
# we can vary b,r,B values only 
# a is generated randomly everytime
def experiment(b,r,B,t,sig_mat_1000,sig_origmat_1000,dict_5shingles,unique_5shingles,H1000):
  # STEP - 1: Split your original document signature vector into b bands of r rows
  sig_mat_arr = np.array(sig_mat_1000).reshape(b,r,-1)

  # STEP - 2: Hash using the same hash functions created for 
      # signature matrix hashing (in the previous cell)

  a = random.sample(list(np.arange(start = 1,stop=r+1,step =1)),r)
  bucket = {}
  for i in range(sig_mat_arr.shape[0]):
    Dict_hash=collections.defaultdict(list)
    for k in range(sig_mat_arr.shape[2]):
      h =np.sum(a*(sig_mat_arr[i,:,k])) % B
      Dict_hash[h].append(files_list[k])
    bucket.update(Dict_hash)

  sig_origmat_1000_arr =np.array(sig_origmat_1000).reshape(b,r,-1)

  # Getting all the candidate items 
  cand_items=set()
  for i in range(sig_origmat_1000_arr.shape[0]):
    h =np.sum(a*(sig_origmat_1000_arr[i,:,0])) % B
    for key,value in bucket.items():
      if key ==h:
        for j in value:
          cand_items.add(j)
  # Type your code here to do the fact check

  k = 5
  #storing shingles in cand_list document wise
  cand_list = list(cand_items)
  print("Count of candidate documents are: ",len(cand_list))
  dict_5shingles_3b = {}
  for file in cand_list:
    dict_5shingles_3b[file] = dict_5shingles[file]


  # Signature matrix: H1000_3b
  sig_mat_1000_3b = sigMatrix(H1000, list(unique_5shingles),dict_5shingles_3b)

  jac_1000_final = filter_jaclist(jac_list_1000,files_list,t)
  # Calculating Jacard Similarity for 1000 hash functions
  jac_list_1000_3b= jaccordsim(cand_list,H1000,sig_mat_1000_3b,sig_origmat_1000)
  print("count of documents with t>0.2 are: ", len(list(jac_1000_final.keys())))
  #filetered documents with jcd>0.2
  jac_1000_final_3b = filter_jaclist(jac_list_1000_3b,cand_list,t)
  print("count of cadidate documents with t>0.2 are: ", len(list(jac_1000_final_3b.keys())))

  ## False Positives
  FP=[]
  for file in cand_list:
    if file in list(jac_1000_final.keys()):
      continue
    else: 
      FP.append(file)
  print("false positive's count: ", len(FP))

  ## False Negatives
  FN=[]
  for file in list(jac_1000_final.keys()):
    if file in list(jac_1000_final_3b.keys()):
      continue
    else: 
      FN.append(file)
  print("false negtive's count: ", len(FN))
  return FP, FN, len(FP),len(FN), list(jac_1000_final.keys()),list(jac_1000_final_3b.keys())

In [ ]:
# Combination 1
b = 25
r = 40
B = 200
t=0.2
x = experiment(b,r,B,t,sig_mat_1000,sig_origmat_1000,dict_5shingles,unique_5shingles,H1000)

Count of candidate documents are:  21
count of documents with t>0.2 are:  8
count of cadidate documents with t>0.2 are:  2
false positive's count:  19
false negtive's count:  6


In [ ]:
# Combination 2
b = 25
r = 40
B = 300
t=0.2
x = experiment(b,r,B,t,sig_mat_1000,sig_origmat_1000,dict_5shingles,unique_5shingles,H1000)

Count of candidate documents are:  21
count of documents with t>0.2 are:  8
count of cadidate documents with t>0.2 are:  4
false positive's count:  17
false negtive's count:  4


In [ ]:
# Combination 3
b = 20
r = 50
B = 400
t=0.2
x = experiment(b,r,B,t,sig_mat_1000,sig_origmat_1000,dict_5shingles,unique_5shingles,H1000)

Count of candidate documents are:  15
count of documents with t>0.2 are:  8
count of cadidate documents with t>0.2 are:  1
false positive's count:  14
false negtive's count:  7


In [ ]:
# Combination 4
b = 20
r = 50
B = 200
t=0.2
x = experiment(b,r,B,t,sig_mat_1000,sig_origmat_1000,dict_5shingles,unique_5shingles,H1000)

Count of candidate documents are:  20
count of documents with t>0.2 are:  8
count of cadidate documents with t>0.2 are:  2
false positive's count:  18
false negtive's count:  6


In [ ]:
# Combination 5
b = 20
r = 50
B = 300
t=0.2
x = experiment(b,r,B,t,sig_mat_1000,sig_origmat_1000,dict_5shingles,unique_5shingles,H1000)

Count of candidate documents are:  17
count of documents with t>0.2 are:  8
count of cadidate documents with t>0.2 are:  2
false positive's count:  15
false negtive's count:  6


In [ ]:
# Combination 6
b = 20
r = 50
B = 400
t=0.2
x = experiment(b,r,B,t,sig_mat_1000,sig_origmat_1000,dict_5shingles,unique_5shingles,H1000)

Count of candidate documents are:  16
count of documents with t>0.2 are:  8
count of cadidate documents with t>0.2 are:  2
false positive's count:  14
false negtive's count:  6


In [ ]:
# Combination 7
b = 10
r = 100
B = 200
t=0.2
x = experiment(b,r,B,t,sig_mat_1000,sig_origmat_1000,dict_5shingles,unique_5shingles,H1000)

Count of candidate documents are:  10
count of documents with t>0.2 are:  8
count of cadidate documents with t>0.2 are:  2
false positive's count:  8
false negtive's count:  6


In [ ]:
# Combination 8
b = 10
r = 100
B = 300
t=0.2
x = experiment(b,r,B,t,sig_mat_1000,sig_origmat_1000,dict_5shingles,unique_5shingles,H1000)

Count of candidate documents are:  9
count of documents with t>0.2 are:  8
count of cadidate documents with t>0.2 are:  1
false positive's count:  8
false negtive's count:  7


In [ ]:
# Combination 9
b = 10
r = 100
B = 400
t=0.2
x = experiment(b,r,B,t,sig_mat_1000,sig_origmat_1000,dict_5shingles,unique_5shingles,H1000)

Count of candidate documents are:  8
count of documents with t>0.2 are:  8
count of cadidate documents with t>0.2 are:  1
false positive's count:  7
false negtive's count:  7


In [ ]:
# Combination 9
b = 10
r = 100
B = 500
t=0.2
x = experiment(b,r,B,t,sig_mat_1000,sig_origmat_1000,dict_5shingles,unique_5shingles,H1000)

Count of candidate documents are:  5
count of documents with t>0.2 are:  8
count of cadidate documents with t>0.2 are:  2
false positive's count:  3
false negtive's count:  6


### References:
1. https://pynative.com/python-random-sample/
2. https://note.nkmk.me/en/python-random-choice-sample-choices/
3. https://github.com/chrisjmccormick/MinHash/blob/master/runMinHashExample.py